In [ ]:
%matplotlib inline

from hydra import initialize, compose
from omegaconf import OmegaConf 

import dr_gen.utils.run as ru
import dr_gen.utils.display as dsp

from dr_gen.analyze.run_group import RunGroup
import dr_gen.analyze.result_plotting as rplt

%load_ext autoreload
%autoreload 2

### Setup Config and Generator

In [ ]:
with initialize(config_path="../configs/", version_base=None):
    cfg = compose(
        config_name="config.yaml",
        overrides=[
            "paths=mac",
        ]
    )

In [ ]:
generator = ru.set_deterministic(cfg.seed)

In [ ]:
print(OmegaConf.to_yaml(OmegaConf.to_container(cfg.paths, resolve=True)))

### Update rplt

In [ ]:
rg = RunGroup()
rg.load_logs_from_base_dir(cfg.paths.agg_results)

### Util Usage

#### Loading and Parsing Run Logs

In [ ]:
# Example: Loading all Runs from Root Dir
run_logs = rp.get_all_logs(cfg.paths.agg_results)
print(f"\nFirst Run Keys: {run_logs[0].keys()}")

In [ ]:
# Example: Parsing a Single Run Log
rcfg, rmets, rmd = rp.parse_run_log(
    run_logs[0]['log_path'],
    run_logs[0]['log_data'],
)
print(f"\nSplits in first run: {list(rmets.keys())}")
print("\nFirst epoch train metrics:")
for k, v in rmets['train'][0].items():
    print(f"  - {k}: {v}")

In [ ]:
# Example: Parsing all runs, dropping error runs
print(f">> Parsing {len(run_logs):,} Runs")
good_runs, bad_runs = rp.parse_run_logs(run_logs)
print(f"   - num good: {len(good_runs):,}")
print(f"   - num w/ errors: {len(bad_runs):,}")

#### Determining the Sweeps in the Data

In [ ]:
# Example: Getting Sweep Info & Simple Display
sweep_info = rp.extract_sweeps(good_runs)
swept_table = dsp.make_table(*rp.get_swept_table(sweep_info['swept_vals']))
print(f"Swept Values:\n{swept_table}")

In [ ]:
# Example: Show the Sweep Combo Run Info
combo_table = dsp.make_table(*rp.get_combo_table_contents(
    sweep_info['combo_key_order'],
    sweep_info['combo_inds'],
))

print(">> First 5 lines of combos")
print(combo_table.get_string(start=0, end=5))

print("\n>> Just Default HPMs")
def filter_function(vals):
    return (
        #vals[0] == "None" and   # Init
        vals[1] == "0.1" and     # LR
        vals[2] == "0.0001" and  # WD
        vals[4] == "270"         # Epochs
    )
print(combo_table.get_string(row_filter=filter_function))

In [ ]:
# Example: Get inds for specific configuration
# Here selected_inds is {(combo_keys): [inds, ...]}
selected_inds = rp.get_inds_by_kvs(
    sweep_info['combo_key_order'], sweep_info['combo_inds'],
    {
        'epochs': '270',
        'model.weights': 'None',
        'optim.lr': '0.1',
    },
)
dsp.make_table(
    [*sweep_info['combo_key_order'], "Count"],
    [[*k, len(selected_inds[k])] for k in selected_inds.keys()],
)

#### Format and Extract Metrics

In [ ]:
all_runs_metrics = rp.remap_run_list_metrics(good_runs)
print(f">> Splits: {list(all_runs_metrics.keys())}")
print(f">> Metrics: {list(all_runs_metrics['train'].keys())}")

In [ ]:
# Example: Get specific metric for a single run
rp.get_run_metrics(all_runs_metrics, "train", "loss", 0)[:5]

In [ ]:
# Example: Get specific metric for list of inds
two_mets = rp.get_runs_metrics(all_runs_metrics, "train", "loss", [0,1])
print(">> Loss for two runs")
dsp.make_table(['Run 0', 'Run 1'], [[two_mets[0][i], two_mets[1][i]] for i in range(4)])

In [ ]:
# Example: Get inds for specific configuration
# Here selected_inds is {(combo_keys): [inds, ...]}
selected_inds = rp.get_inds_by_kvs(
    sweep_info['combo_key_order'], sweep_info['combo_inds'],
    {
        'epochs': '270',
        #'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    },
)
print(dsp.make_table(
    [*sweep_info['combo_key_order'], "Count"],
    [[*k, len(selected_inds[k])] for k in selected_inds.keys()],
))
selected_metrics = rp.get_selected_run_metrics(
    all_runs_metrics,
    "train",
    "loss",
    selected_inds,
)
for k, v in selected_metrics.items():
    print(f">> Selected values: {k}")
    print(f"  - Num Runs: {len(v)}")
    print(f"  - Epochs Per Run: {len(v[0])}")
    print(f"  - First few metric vals: {v[0][:6]}")
    print()

#### Basic Plotting Utils

In [ ]:
# Example: Plot multiple splits for same run
pu.plot_lines(
    pu.get_plt_cfg(
        ylabel="acc", title="Accuracy During Training",
        ylim=(70, 100), labels=['train', 'val']
    ),
    [
        rp.get_run_metrics(all_runs_metrics, "train", "acc1", 0),
        rp.get_run_metrics(all_runs_metrics, "val", "acc1", 0),
    ],
)

In [ ]:
# Example: Plot one split for multiple inds (manually selected)
pu.plot_lines(
    pu.get_plt_cfg(
        ylabel="acc", title="Val Accuracy During Training",
        ylim=(70, 86), labels=[0,1,2],
    ),
    rp.get_runs_metrics(all_runs_metrics, "val", "acc1", [0,1,2]),
)

In [ ]:
# Example: Plot one split for multiple inds (selected via combo)
selected_inds = rp.get_inds_by_kvs(
    sweep_info['combo_key_order'], sweep_info['combo_inds'],
    {
        'epochs': '270',
        'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    },
)
selected_metrics = list(rp.get_selected_run_metrics(
    all_runs_metrics,
    "val",
    "acc1",
    selected_inds,
).items())
met_keys = [sm[0] for sm in selected_metrics]
met_values = [sm[1] for sm in selected_metrics]
pu.plot_lines(
    pu.get_plt_cfg(
        ylabel="acc",
        title=rplt.kvs_to_str(
            list(zip(sweep_info['combo_key_order'], met_keys[0]))
        ),
        ylim=(70, 86),
        legend=False,
        #labels=[str(k) for k in selected_metrics],
    ),
    met_values[0],
)

#### Simple Histogram and Ind Data

In [ ]:
# Example of getting aggregate stats and vals for a single ind
all_kvs, all_split_vals, _ = rp.get_selected_combo(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    splits=['train', 'val'],
    ignore_keys=["optim.step_size", "epochs"],
    num_seeds=5,
)
rd_stats_ind100 = pu.get_runs_data_stats_ind(
    all_split_vals['val'][0],
    ind=100,
)
rd_stats_ind100

In [ ]:
all_kvs, all_split_vals, _ = rp.get_selected_combo(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    splits=['train', 'val'],
    ignore_keys=["optim.step_size", "epochs"],
)
rd_stats_ind100 = pu.get_runs_data_stats_ind(
    all_split_vals['val'][0],
    ind=100,
)
hp.plot_histogram(
    pu.get_plt_cfg(
        nbins=rd_stats_ind100['n']//4,
        hist_range=(80, 90),
        title=f"Accuracy Distribution, {rd_stats_ind100['n']} Seeds",
        ylabel="Num Runs",
    ),
    rd_stats_ind100['vals'],    
)

In [ ]:
all_kvs, all_split_vals, _ = rp.get_selected_combo(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        #'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    splits=['train', 'val'],
    ignore_keys=["optim.step_size", "epochs"],
)
stats1_ind100 = pu.get_runs_data_stats_ind(
    all_split_vals['val'][0],
    ind=100
)
stats2_ind100 = pu.get_runs_data_stats_ind( 
    all_split_vals['val'][1],
    ind=100,
)
labels_kvstrs = [
    rplt.kvs_to_str(
        [(k, v) for k, v in kvs if k == "model.weights"]
    ) for kvs in all_kvs
]
title_kvstr = rplt.kvs_to_str(
    [(k, v) for k, v in all_kvs[0] if k != "model.weights"]
)
ns = [stats1_ind100['n'], stats2_ind100['n']]
hp.plot_histogram_compare(
    pu.get_plt_cfg(
        nbins=max(ns)//4,
        hist_range=(80, 90),
        title=f"Accuracy Distribution | {title_kvstr}",
        ylabel="Num Runs",
        labels=[
            rplt.kvs_to_str(
                [(k, v) for k, v in kvs if k == "model.weights"]
            ) + f" n={ns[i]} " for i, kvs in enumerate(all_kvs)
        ],
    ),
    [
        stats1_ind100,
        stats2_ind100,
    ],
)

In [ ]:
all_kvs, all_split_vals, _ = rp.get_selected_combo(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        #'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    splits=['train', 'val'],
    ignore_keys=["optim.step_size", "epochs"],
)
stats1_ind100 = pu.get_runs_data_stats_ind(
    all_split_vals['val'][0],
    ind=100
)
stats2_ind100 = pu.get_runs_data_stats_ind( 
    all_split_vals['val'][1],
    ind=100,
)
results = ks.calculate_ks_for_run_sets(
    stats1_ind100['vals'],
    stats2_ind100['vals'],
)
pu.plot_cdf(
    pu.get_plt_cfg(),
    results['all_vals'],
    results['cdf1'],
)
pu.plot_cdfs(
    pu.get_plt_cfg(
        labels=[
            rplt.kvs_to_str(
                [(k, v) for k, v in kvs if k == "model.weights"]
            ) + f" n={ns[i]} " for i, kvs in enumerate(all_kvs)
        ],
    ),
    results['all_vals'],
    [
        results['cdf1'],
        results['cdf2'],
    ]
)

#### Using Result Plotting Utils

In [ ]:
rplt.plot_run_splits(
    good_runs,
    all_runs_metrics,
    sweep_info,
    run_ind=0,
    ignore_keys=['optim.step_size', 'epochs'],
    ylim=(75, 100),
)

In [ ]:
rplt.plot_split_summaries(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    splits=['train', 'val'],
    ignore_keys=["optim.step_size", "epochs"],
    num_seeds=None,
    ylim=(82, 86),
)
    

In [ ]:
rplt.plot_split_summaries(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    splits=['train', 'val'],
    ignore_keys=["optim.step_size", "epochs"],
    num_seeds=20,
    ylim=(82, 86),
)
    

In [ ]:
rplt.plot_combo_histogram(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    split='val',
    epoch=110,
    metric='acc1',
    ignore_keys=["optim.step_size", "epochs"],
    num_seeds=None,
    #nbins=10,
    hist_range=(82,86),
    density=True,
)

In [ ]:
rplt.plot_combo_histogram_compare(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        #'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    split="val",
    epoch=100,
    metric="acc1",
    ignore_keys=["optim.step_size", "epochs"],
    num_seeds=20,
    nbins=10,
    vary_key="model.weights",
)

In [ ]:
rplt.ks_stats_plot_cdfs(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        #'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    split="val",
    epoch=100,
    metric="acc1",
    ignore_keys=["optim.step_size", "epochs"],
    num_seeds=None,
    vary_key="model.weights",
)

In [ ]:
rplt.ks_stats_plot_cdfs(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        'model.weights': 'None',
        #'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    split="val",
    epoch=100,
    metric="acc1",
    ignore_keys=["optim.step_size", "epochs"],
    num_seeds=None,
    vary_key="optim.lr",
    vary_vals=[str(v) for v in [0.1, 0.01]],
)

In [ ]:
rplt.ks_stat_plot_cdfs_histograms(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        #'model.weights': 'None',
        'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    split="val",
    epoch=100,
    metric="acc1",
    ignore_keys=["optim.step_size", "epochs"],
    num_seeds=None,
    vary_key="model.weights",
    nbins=40,
)

In [ ]:
rplt.ks_stat_plot_cdfs_histograms(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        'model.weights': 'None',
        #'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    split="val",
    epoch=100,
    metric="acc1",
    ignore_keys=["optim.step_size", "epochs"],
    num_seeds=20,
    vary_key="optim.lr",
    vary_vals=[str(lr) for lr in [0.1, 0.01]],
    nbins=40,
)

In [ ]:
rplt.ks_stat_plot_cdfs_histograms(
    good_runs,
    all_runs_metrics,
    sweep_info,
    kv_select={
        'epochs': '270',
        'model.weights': 'None',
        #'optim.lr': '0.1',
        'optim.weight_decay': '0.0001',
    }, 
    split="val",
    epoch=100,
    metric="acc1",
    ignore_keys=["optim.step_size", "epochs"],
    num_seeds=20,
    vary_key="optim.lr",
    vary_vals=[str(lr) for lr in [0.1, 0.04]],
    nbins=40,
)